## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-27-20-51-10 +0000,nypost,Homeless man dies after being found unconsciou...,https://nypost.com/2025/07/27/us-news/homeless...
1,2025-07-27-20-50-35 +0000,nyt,U.S. Reaches Preliminary Trade Deal With Europe,https://www.nytimes.com/2025/07/27/world/europ...
2,2025-07-27-20-44-23 +0000,nyt,R.S.F. Coalition Names Leaders of Sudan’s Para...,https://www.nytimes.com/2025/07/27/world/afric...
3,2025-07-27-20-44-00 +0000,wsj,President Trump said he reached a trade agreem...,https://www.wsj.com/economy/trade/trump-says-h...
4,2025-07-27-20-40-22 +0000,nypost,"Jamaica, Queens tries to rebrand with $100K ma...",https://nypost.com/2025/07/27/us-news/jamaica-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,30
128,gaza,17
132,aid,15
200,israel,10
1,dies,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
61,2025-07-27-17-13-12 +0000,nypost,Trump says ‘Israel is gonna have to make a dec...,https://nypost.com/2025/07/27/us-news/trump-sa...,80
138,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...,76
151,2025-07-27-03-14-00 +0000,wsj,Thailand and Cambodia Consider Cease-Fire Afte...,https://www.wsj.com/world/asia/thailand-and-ca...,67
146,2025-07-27-04-07-59 +0000,nyt,Trump Says He Expects Quick Cease-Fire Between...,https://www.nytimes.com/2025/07/26/world/asia/...,65
54,2025-07-27-17-42-12 +0000,nyt,"At Trump’s Urging, Thailand and Cambodia to Me...",https://www.nytimes.com/2025/07/26/world/asia/...,64


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
61,80,2025-07-27-17-13-12 +0000,nypost,Trump says ‘Israel is gonna have to make a dec...,https://nypost.com/2025/07/27/us-news/trump-sa...
138,46,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...
17,31,2025-07-27-20-20-02 +0000,nypost,GOP makes Zohran Mamdani a key campaign issue ...,https://nypost.com/2025/07/27/us-news/ny-repub...
15,31,2025-07-27-20-22-15 +0000,nypost,Armed hero praised for confronting Michigan Wa...,https://nypost.com/2025/07/27/us-news/armed-he...
56,29,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...
165,27,2025-07-27-00-00-00 +0000,wsj,Americans view President Trump’s tax-and-spend...,https://www.wsj.com/politics/policy/trump-wsj-...
40,25,2025-07-27-18-59-26 +0000,nypost,NYC air quality plummets as Canadian wildfire ...,https://nypost.com/2025/07/27/us-news/nyc-air-...
90,25,2025-07-27-12-35-44 +0000,wapo,Landlord convicted of killing Palestinian Amer...,https://www.washingtonpost.com/nation/2025/07/...
32,24,2025-07-27-19-27-59 +0000,nypost,"3 dead in small plane crash in Monterey, Calif...",https://nypost.com/2025/07/27/us-news/3-dead-i...
27,24,2025-07-27-19-42-11 +0000,nypost,90 prosecutors quit Nassau County DA’s Office ...,https://nypost.com/2025/07/27/us-news/90-prose...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
